In [1]:
%display latex

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os

In [3]:
#Define space-time manifold and coordinates

n = 3
M = Manifold(1+n, 'M', structure='Lorentzian')
X.<t,r,th,ph> = M.chart(r"t r:(0,+oo) th:(0,pi):\theta ph:(0,2*pi):\phi")

In [4]:
eps = var('eps', latex_name='\\varepsilon')

h = function('h')(r,th)
k = function('k')(r,th)
m = function('m')(r,th)
omega = function('omega', latex_name="\\omega")(r,th)

nu = function('nu', latex_name='\\nu')(r)
lamb = function('lamb', latex_name='\\lambda')(r)

Omegae = function('Omegaevalue', latex_name = "\\Omega_e")(eps)
Ne = function('Ne', latex_name = "N_e")(r,eps)

In [ ]:
gdd = M.tensor_field(0,2, name='gdd')
gdd[0,0] = -exp(nu)
gdd[1,1] = exp(lamb)
gdd[2,2] = r^2
gdd[3,3] = r^2*sin(th)^2

K1 = M.tensor_field(0,2, name='K1')
K1[0,3] = omega*r^2*sin(th)^2
K1[3,0] = omega*r^2*sin(th)^2

K2 = M.tensor_field(0,2, name='K2')
K2[0,0] = -4*exp(nu)*h + 2*r^2*sin(th)^2*omega^2
K2[1,1] = 4*exp(lamb)*m
K2[2,2] = 4*k*r^2
K2[3,3] = 4*k*r^2*sin(th)^2

gedd = M.metric()
for i in range(0,n+1):
    for j in range(0,n+1):
        gedd[i,j] = gdd[i,j] + eps*K1[i,j] + 1/2*eps^2*K2[i,j]

geuu = gedd.inverse()
ricci = gedd.ricci()
s_curv = geuu['^{ab}']*ricci['_ab']
eins = ricci - gdd*s_curv/2                                                                             #G_\mu\nu e

deins = M.tensor_field(0,2,'deins')
ddeins = M.tensor_field(0,2,'ddeins')
for i in range(0,n+1):
    for j in range(0,n+1):
        deins[i,j] = diff(eins[i,j].expr(),eps)
        ddeins[i,j] = diff(eins[i,j].expr(),eps,eps)

In [ ]:
show(LatexExpr("g_{\\varepsilon}\\rvert_{\\varepsilon=0} = "), gdd[:])
show(LatexExpr("K_1 = "), K1[:])
show(LatexExpr("K_2 = "), K2[:])
show(LatexExpr("g_{\\varepsilon} = "), gedd[:])

In [ ]:
#4-velocities of the fluid for any epsilon

ue = M.vector_field(name="ue")
ue[0] = Ne; ue[1] = 0; ue[2] = 0; ue[3] = Ne*Omegae

u = M.vector_field(name="u")
u[0] = exp(-nu/2); u[1] =  0; u[2] =  0; u[3] =  0

Nevalue = solve(u[0].expr()==ue[0].expr(), Ne)[0].rhs()                                                    #Eq 4.20
Omegaevalue = solve(u[3].expr()==ue[3].expr(), Omegae)[0].rhs()                                            #Eq 4.20

In [ ]:
show(LatexExpr("N_{\\varepsilon}\\rvert_{\\varepsilon=0} = "), Nevalue)
show(LatexExpr("\\Omega_{\\varepsilon}\\rvert_{\\varepsilon=0} = "), Omegaevalue)

In [ ]:
u1 = M.vector_field(name="u1",latex_name="u^{(1)}")
u2 = M.vector_field(name="u2",latex_name="u^{(2)}")

u10 = function('u10',latex_name="u^{(1)0}")(r)
u20 = function('u20',latex_name="u^{(2)0}")(r)

Omega1 = function('Omega1', latex_name="\\Omega^{(1)}")(eps)
Omega2 = function('Omega2', latex_name="\\Omega^{(2)}")(eps)

s0 = eps == 0
s1 = Ne.subs(eps==0) == Nevalue
s2 = Omegae.subs(eps==0) == Omegaevalue
s3 = diff(Ne,eps).subs(eps==0) == u10
s4 = diff(Omegae,eps).subs(eps==0) == Omega1
s5 = diff(Ne,eps,eps).subs(eps==0) == u20
s6 = diff(Omegae,eps,eps).subs(eps==0) == Omega2

sustituciones = (s0,s1,s2,s3,s4,s5,s6)

for i in range(0,n+1):
    u1[i] = diff(ue[i].expr(),eps).subs(sustituciones)
    u2[i] = diff(ue[i].expr(),eps,eps).subs(sustituciones)

Ue = ue.down(gedd)
geUeue = Ue['_a']*ue['^a']
dgeUeue = diff(geUeue.expr(), eps).subs(sustituciones) == 0
u10value = solve(dgeUeue, u10)[0].rhs()
s7 = u10 == u10value
u1.apply_map(lambda cmp: cmp.subs(u10==u10value))                                                          #Eq 4.21

ddgeUeue = diff(geUeue.expr(), eps, eps).subs(sustituciones) == 0
u20value = solve(ddgeUeue, u20)[0].rhs()
s8 = u20 == u20value
u2.apply_map(lambda cmp: cmp.subs(u20==u20value).subs(u10==u10value))                                      #Eq 4.22

In [ ]:
show(u1.display())
show(u2.display())

In [ ]:
deins.apply_map(lambda cmp: cmp.subs(sustituciones))                                             #d/de(G_\mu\nu)|e=0
ddeins.apply_map(lambda cmp: cmp.subs(sustituciones))                                        #d^2/de^2(G_\mu\nu)|e=0

In [ ]:
Ee = function('Ee', latex_name="E_{\\varepsilon}")(r,eps)
Pe = function('Pe', latex_name="P_{\\varepsilon}")(r,eps)

eins0 = M.tensor_field(0,2, 'eins0', latex_name='G_0')
Tedd = M.tensor_field(0,2,name='Tedd',latex_name="Tdd_{\\varepsilon}")
Tdd = M.tensor_field(0,2, 'Tdd')
for i in range(0,n+1):
    for j in range(0,n+1):
        eins0[i,j] = eins[i,j]
        Tedd[i,j] = (Ee + Pe)*Ue[i]*Ue[j] + Pe*gedd[i,j]                                                 #T_\mu\nu e
        Tdd[i,j] = Tedd[i,j]

eins0.apply_map(lambda cmp: cmp.subs(sustituciones))                                                       #G_\mu\nu
Tdd.apply_map(lambda cmp: cmp.subs(sustituciones))                                                         #T_\mu\nu

In [ ]:
show(LatexExpr("T_{\\varepsilon}\\rvert_{\\varepsilon=0} = "), Tdd[:])

In [ ]:
kappa = 8*pi
E = function('E')(r)
P = function('P')(r)

s9 = Ee.subs(eps==0) == E
s10 = Pe.subs(eps==0) == P
sustituciones = (s1,s2,s3,s4,s5,s6,s7,s8,s9,s10)
Tdd.apply_map(lambda cmp: cmp.subs(sustituciones))
E_tt = solve(eins0[0,0].expr()==kappa*Tdd[0,0].expr(),E)[0].rhs()                                            #Eq 3.7
P_rr = solve(eins0[1,1].expr()==kappa*Tdd[1,1].expr(),P)[0].rhs()                                            #Eq 3.8
P_thth = solve(eins0[2,2].expr()==kappa*Tdd[2,2].expr(),P)[0].rhs()
P_phph = solve(eins0[3,3].expr()==kappa*Tdd[3,3].expr(),P)[0].rhs()
ddnu = solve(P_rr == P_thth, diff(nu,r,r))[0].rhs()                                                          #Eq 3.9

In [ ]:
show(LatexExpr("E = "), E_tt)
show(LatexExpr("P_{rr} = "), P_rr)
show(LatexExpr("\\nu'' = "), ddnu)

In [ ]:
dlamb = solve(E==E_tt,diff(lamb,r))[0].rhs()                                                                 #Eq 3.10
dnu = solve(P==P_rr,diff(nu,r))[0].rhs()                                                                     #Eq 3.11

In [ ]:
show(LatexExpr("\\lambda' = "), dlamb)
show(LatexExpr("\\nu' = "), dnu)

In [ ]:
h0 = function('h0', latex_name="h_0")(r,th)
k0 = function('k0', latex_name="k_0")(r,th)
m0 = function('m0', latex_name="m_0")(r,th)

h2 = function('h2', latex_name="h_2")(r,th)
k2 = function('k2', latex_name="k_2")(r,th)
m2 = function('m2', latex_name="m_2")(r,th)

m = m0 + m2*(3*cos(th)^2-1)/2
k = k0 + k2*(3*cos(th)^2-1)/2
h = h0 + h2*(3*cos(th)^2-1)/2

In [ ]:
E1 = function('E1', latex_name='E^{(1)}')(r)
E2 = function('E2', latex_name='E^{(2)}')(r)
P1 = function('P1', latex_name='P^{(1)}')(r)
P2 = function('P2', latex_name='P^{(2)}')(r)

s11 = diff(Ee,eps).subs(eps==0) == E1
s12 = diff(Ee,eps,eps).subs(eps==0) == E2
s13 = diff(Pe,eps).subs(eps==0) == P1
s14 = diff(Pe,eps,eps).subs(eps==0) == P2

sustituciones = (s0,s1,s2,s3,s4,s5,s6,s7,s8,s9,s10,s11,s12,s13,s14)

In [ ]:
dTdd = M.tensor_field(0,2,'dTdd')
ddTdd = M.tensor_field(0,2,'ddTdd')
for i in range(0,n+1):
    for j in range(0,n+1):
        dTdd[i,j] = diff(Tedd[i,j].expr(),eps)
        ddTdd[i,j] = diff(Tedd[i,j].expr(),eps,eps)
dTdd.apply_map(lambda cmp: cmp.subs(sustituciones))                                              #d/de(T_\mu\nu)|e=0
ddTdd.apply_map(lambda cmp: cmp.subs(sustituciones))                                         #d^2/de^2(T_\mu\nu)|e=0

In [ ]:
# Calculamos ahora la primera derivada de la ecuación

show(deins[1,1],LatexExpr(" = "), kappa*dTdd[1,1])
show(deins[2,2],LatexExpr(" = "), kappa*dTdd[2,2])
show(deins[3,3],LatexExpr(" = "), kappa*dTdd[3,3])

P1 = solve(deins[1,1].expr() == kappa*dTdd[1,1].expr(),P1)[0].rhs()

show(LatexExpr("\\text{De aquí sacamos }P^{1}(r)\\rightarrow\\text{$P^1(r)$ = }"), P1)

In [ ]:
show(deins[0,0],LatexExpr(" = "), kappa*dTdd[0,0])

E1 = solve(deins[0,0].expr() == kappa*dTdd[0,0].expr(),E1)[0].rhs()
E1 = E1.subs(sustituciones)
show(LatexExpr("\\text{De aquí sacamos }E^{1}(r)\\rightarrow\\text{$E^1(r)$ = }"), E1)

In [ ]:
show(deins[0,3].expr() == kappa*dTdd[0,3].expr())                                                           #Eq 48

In [ ]:
# Para la segunda derivada tenemos

show(ddeins[0,0].expr(),LatexExpr(" = "), kappa*ddTdd[0,0].expr())

In [ ]:
show(ddeins[1,1],LatexExpr(" = "), kappa*ddTdd[1,1])

In [ ]:
show(ddeins[2,2],LatexExpr(" = "), kappa*ddTdd[2,2])

In [ ]:
show(ddeins[3,3],LatexExpr(" = "), kappa*ddTdd[3,3])

In [ ]:
show(ddeins[0,3],LatexExpr(" = "), kappa*ddTdd[0,3])

In [ ]:
show(ddeins[1,2],LatexExpr(" = "), kappa*ddTdd[1,2])

In [ ]:
# Aplicamos barotropic EoS
# Utilizando (6.11) de Reina con P1=0 y E1=0
E2*diff(P,r) - P2*diff(E,r) == 0

In [ ]:
fm(r) = m0 + m2*(3*cos(th)^2-1)/2
fk(r) = k0 + k2*(3*cos(th)^2-1)/2
fh(r) = h0 + h2*(3*cos(th)^2-1)/2

ddeins.apply_map(lambda cmp: cmp.substitute_function(function('h'), fh))
ddeins.apply_map(lambda cmp: cmp.substitute_function(function('m'), fm))
ddeins.apply_map(lambda cmp: cmp.substitute_function(function('k'), fk))